# Wide and CNN
Google 에서 발표현 Wide and Deep 을 약간 변형한 형태로 Wide(Logistic)과 Cnn(Deep)을 Combine 형태의 Network 를 개발하였다. <br> 
여기서는 자연어를 Wide and Cnn 을 사용하여 Classification 하는 것을 테스트 하고자 한다. 

In [80]:
import requests
import json, os

nn_id = 'nnnnn995'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


# 네트워크 및 그래프 버전 생성 


In [81]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nnnnn995
evaluation result : nnnnn995


# 그래프 플로우 정의

In [82]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'wcnn'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : wcnn


# 데이터 업로드

In [83]:
import os

return_dict = {}
return_dict['test'] = open('../../data/seq2seq_several_case.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# 데이터 노드 정의

In [84]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/data_node/',
                      json={
                          "preprocess":  "none",
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nnnnn995/1/data_node


# 데이터 Feeder 정의

In [85]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/wcnn/nnid/'+nn_id+'/ver/1/node/pre_feed_train/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "channel" : 1,
                         "encode_len" : 6,
                         "preprocess": "none",
                         "vocab_size" : 100,
                         "lable_size" : 22, #총 label 수에서 3개 더해야함 
                         "embed_type" : 'onehot'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'channel': 1, 'lable_size': 15, 'vocab_size': 100, 'decode_column': 'decode', 'embed_type': 'onehot', 'encode_column': 'encode', 'preprocess': 'none', 'encode_len': 6}


# 네트워크 정의

In [86]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/wcnn/nnid/'+nn_id+'/ver/1/node/netconf_node/',
                     json={
                         "param":{"epoch": 200
                                  ,"traincnt": 1
                                  ,"batch_size":64
                                  ,"predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                    "layeroutputs":32,
                                    "net_type":"cnn",
                                    "eval_type":"category",
                                    "optimizer":"AdamOptimizer" #RMSPropOptimizer
                                     }
                         ,"layers": [
                                    {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.5",
                                     "layercnt":2
                                    },
                                    {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [1, 1],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.5",
                                     "layercnt":1
                                    }
                                   ]
                         ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                         ,"labels":[]
                        })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'out': {'node_out': 625, 'padding': 'SAME', 'active': 'softmax'}, 'config': {'eval_type': 'category', 'learnrate': 0.001, 'net_type': 'cnn', 'num_classes': 10, 'optimizer': 'AdamOptimizer', 'layeroutputs': 32}, 'model_path': '/hoya_model_root/nnnnn995/1/netconf_node', 'layers': [{'maxpoolstride': [2, 2], 'layercnt': 2, 'cnnfilter': [3, 3], 'padding': 'SAME', 'active': 'relu', 'cnnstride': [1, 1], 'droprate': '0.5', 'maxpoolmatrix': [2, 2]}, {'maxpoolstride': [2, 2], 'layercnt': 1, 'cnnfilter': [3, 3], 'padding': 'SAME', 'active': 'relu', 'cnnstride': [1, 1], 'droprate': '0.5', 'maxpoolmatrix': [1, 1]}], 'param': {'epoch': 200, 'traincnt': 1, 'batch_size': 64, 'predictcnt': 10}, 'labels': []}


# 테스트 프로세스 정의

In [87]:
# upload files for test
return_dict = {}
return_dict['test'] = open('../../data/seq2seq_several_case.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 소스 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 전처리 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                      json={
                          "preprocess":  "none",
                      })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 저장 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/test_data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


# Feeder 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/wcnn/nnid/'+nn_id+'/ver/1/node/pre_feed_test/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "channel" : 1,
                         "encode_len" : 22,
                         "preprocess": "",
                         "vocab_size" : 100,
                         "embed_type" : 'onehot',
                         "lable_size" : 22
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

node_name = 'eval_node'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/eval/nnid/'+nn_id+'/ver/1/node/'+node_name+'/',
                    json={
                        "type": "category",
                    })

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']
evaluation result : {'type': 'csv', 'source_type': 'local', 'source_sql': 'all', 'multi_node_flag': None, 'predict_path': '/hoya_src_root/nnnnn995/1/predict', 'source_parse_type': 'raw', 'source_path': '/hoya_src_root/nnnnn995/1/test_data_node', 'source_server': 'local', 'max_sentence_len': 0}
evaluation result : none
evaluation result : /hoya_str_root/nnnnn995/1/test_data_node
evaluation result : {'channel': 1, 'lable_size': 22, 'vocab_size': 100, 'decode_column': 'decode', 'embed_type': 'onehot', 'encode_column': 'encode', 'preprocess': '', 'encode_len': 15}
evaluation result : {'type': 'category'}


# Train

In [88]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'status': '404', 'result': "error on eval wcnn : object of type 'NoneType' has no len()"}


# Predict

In [92]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] [직급]" }
                     )
data = json.loads(resp.json())
print("evaluation result(1) : {0}".format(data)) 

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] [직책]" }
                     )
data = json.loads(resp.json())
print("evaluation result(1) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[회사] [이름]" }
                     )
data = json.loads(resp.json())
print("evaluation result(1) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] [사진]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(9) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [담당임원]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(4) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [휴대전화]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(11) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [교대근무조]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] [직번]은" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(12) : {0}".format(data))


resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [직급]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[회사]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[업무]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(7) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/wcnn/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[근태코드]" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(8) : {0}".format(data))

evaluation result(1) : [1]
evaluation result(1) : [1]
evaluation result(1) : [1]
evaluation result(9) : [9]
evaluation result(4) : [4]
evaluation result(11) : [11]
evaluation result(2) : [2]
evaluation result(12) : [12]
evaluation result(2) : ['-1']
evaluation result(2) : ['-1']
evaluation result(7) : [7]
evaluation result(8) : [8]
